In [1]:
import cv2
import numpy as np
import os

In [2]:
name = input("Enter Name\n")

Enter Name
Sree


In [3]:
if not os.path.exists('data/Class/%s'%name):
    os.makedirs('data/Class/%s'%name)

In [4]:
#function to detect face using OpenCV
def detect_face(img):
    #convert the test image to gray image as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #load OpenCV face detector, I am using haarcascade_frontalface_default
    
    face_cascade = cv2.CascadeClassifier('data/haarcascade_frontalface_default.xml')
    #let's detect multiscale (some images may be closer to camera than others) images
    #result is a list of faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    
    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None
    
    #under the assumption that there will be only one face,
    #extract the face area
    (x, y, w, h) = faces[0]
    
    #return only the face part of the image
    return img[y:y+w, x:x+h], faces[0]

In [5]:
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 255, 255), 2)

In [6]:
video_capture = cv2.VideoCapture(0)
counter = 1
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    print(frame)
    print(type(frame))
    print(ret)

    face,rect = detect_face(frame)
    if rect is not None:
        draw_rectangle(frame,rect)
        cv2.imwrite('data/Class/%s/%s.jpg'%(name,counter),face)
        counter+=1
    # Display the resulting frame
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break
# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

[[[ 99  59  42]
  [100  60  43]
  [102  59  43]
  ...
  [ 79  41   6]
  [ 79  44  13]
  [ 83  48  18]]

 [[ 98  59  45]
  [ 98  59  45]
  [ 97  57  43]
  ...
  [ 79  41   6]
  [ 78  42  12]
  [ 86  51  20]]

 [[ 96  60  45]
  [ 92  56  42]
  [ 94  56  42]
  ...
  [ 80  43   6]
  [ 80  46  12]
  [ 82  48  15]]

 ...

 [[ 26   0   0]
  [ 26   0   0]
  [ 24   0   0]
  ...
  [ 18   0   0]
  [ 22   0   0]
  [ 22   0   0]]

 [[ 26   0   0]
  [ 26   0   0]
  [ 24   0   0]
  ...
  [ 18   0   0]
  [ 20   0   0]
  [ 20   0   0]]

 [[ 24   0   0]
  [ 24   0   0]
  [ 24   0   0]
  ...
  [ 18   0   0]
  [ 18   0   0]
  [ 18   0   0]]]
<class 'numpy.ndarray'>
True
[[[ 99  63  48]
  [ 98  62  47]
  [ 95  59  42]
  ...
  [ 82  48   0]
  [ 84  56  10]
  [ 87  59  14]]

 [[ 97  62  45]
  [ 97  62  45]
  [100  67  45]
  ...
  [ 86  51   7]
  [ 84  57  16]
  [ 86  59  18]]

 [[ 97  62  45]
  [ 99  64  47]
  [103  68  49]
  ...
  [ 87  50  13]
  [ 86  55  19]
  [ 88  57  21]]

 ...

 [[ 28   0   0]
  [ 28 

In [7]:
def name_map(x):
    if x=='Sachin':
        return 0
    if x=='Sree':
        return 1

# Training

In [8]:
def prepare_training_data(data_folder_path):
    
    #------STEP-1--------
    #get the directories (one directory for each subject) in data folder
    dirs = os.listdir(data_folder_path)
    
    #list to hold all subject faces
    faces = []
    #list to hold labels for all subjects
    labels = []
    
    #let's go through each directory and read images within it
    for dir_name in dirs:
        
            
        #------STEP-2--------
        #extract label number of subject from dir_name
        label = dir_name
        
        #build path of directory containin images for current subject subject
        #sample subject_dir_path = "data/DB/sachin"
        subject_dir_path = data_folder_path + "/" + dir_name
        
        #get the images names that are inside the given subject directory
        subject_images_names = os.listdir(subject_dir_path)
        
        #------STEP-3--------
        #go through each image name, read image, 
        #detect face and add face to list of faces
        for image_name in subject_images_names:
            
            #ignore system files like .DS_Store
            if image_name.startswith("."):
                continue;
            
            #build image path
            #sample image path = training-data/s1/1.pgm
            image_path = subject_dir_path + "/" + image_name

            #read image
            image = cv2.imread(image_path)
            
            #display an image window to show the image 
            cv2.imshow("Training on image...", image)
            cv2.waitKey(100)

            
            
            #------STEP-4--------
            #for the purpose of this tutorial
            #we will ignore faces that are not detected
            if image is not None:
                #add face to list of faces
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                faces.append(image)
                #add label for this face
                labels.append(name_map(label))
            
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    return faces, labels

In [9]:
print("Preparing data...")
faces, labels = prepare_training_data("data/Class")
print("Data prepared")

#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

Preparing data...
Data prepared
Total faces:  29
Total labels:  29


In [10]:
np.array(labels)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1])

In [11]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

face_recognizer.train(faces, np.array(labels))

In [12]:
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 255, 255), 2)

def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 255, 255), 2)

In [13]:
def get_name(x):
    if x==0:
        return 'Sachin'
    if x==1:
        return 'Sree'
    if x == 2:
        return 'messi'

In [14]:
def predict(test_img):
    #make a copy of the image as we don't want to chang original image
    img = test_img.copy()
    #detect face from the image
    face, rect = detect_face(img)
    print(rect)

    #predict the image using our face recognizer 
    try:
        label= face_recognizer.predict(face)
        print(label)
        #get name of respective label returned by face recognizer
        #label_text = subjects[label[0]]
        label_text = get_name(label[0])
        #draw a rectangle around face detected
        draw_rectangle(img, rect)
        #draw name of predicted person
        draw_text(img, label_text, rect[0], rect[1]-5)
    except:
        pass
    
    return img

In [15]:
video_capture = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    predicted_img1 = predict(frame)

    # Display the resulting frame
    cv2.imshow('Video', predicted_img1)

    if cv2.waitKey(1) & 0xFF == 27:
        break
# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

[271  83 271 271]
[274  87 264 264]
[264  79 282 282]
[269  87 268 268]
[268  83 276 276]
[287 102 241 241]
